In [ ]:
ls instrument_model

In [ ]:
!head instrument_model/cmbs4_instrument_model.tbl


In [ ]:
from astropy.table import QTable, Table
import numpy as np

In [ ]:
baseline_design = QTable.read('instrument_model/cmbs4_instrument_model.tbl',  format='ascii.ipac')
baseline_design.add_index("band")

In [ ]:
reference_design = Table.read("../202006_foregrounds_extragalactic_cmb_tophat/cmbs4_tophat.csv")
reference_design.add_index("band")

In [ ]:
baseline_design

In [ ]:
reference_design

In [ ]:
baseline_design["NET_ratio"] = np.nan
baseline_design["num_detectors_ratio"] = np.nan
baseline_design["efficiency_ratio"] = 1 / 0.2 # Reference design efficiency

In [ ]:
for row in baseline_design:
    if "P" in row["band"]:
        row["NET_ratio"] = row["NET"].value / reference_design.loc[row["band"].replace("P","")]["pole_NET_uKsqrts"]
        row["num_detectors_ratio"] = row["detectors_per_tube"] / reference_design.loc[row["band"].replace("P","")]["detectors_per_tube"]
    else:
        row["NET_ratio"] = row["NET"].value / reference_design.loc[row["band"]]["NET_uKsqrts"]
        row["num_detectors_ratio"] = row["detectors_per_tube"] / reference_design.loc[row["band"]]["detectors_per_tube"]
baseline_design["num_detectors_ratio"][baseline_design["telescope"] == "LAT"] *= 4 # from 4 wafers to 1 wafer per tube

In [ ]:
baseline_design

In [ ]:
import toml

In [ ]:
efficiencies = toml.load("./s4_baseline_design.toml")["experiment"]

In [ ]:
efficiencies["sensitivity_factor"]

In [ ]:
efficiencies["observing_efficiency"]

In [ ]:
for row in baseline_design:
    site = "Pole" if ("P" in row["band"] or row["telescope"] == "SAT") else "Chile"
    row["efficiency_ratio"] *= efficiencies["observing_efficiency"][site][row["telescope"]][row["band"]]
    row["efficiency_ratio"] *= efficiencies["sensitivity_factor"][site][row["telescope"]].get(row["band"], efficiencies["sensitivity_factor"][site][row["telescope"]].get("default", np.nan))

In [ ]:
t = baseline_design[["band", "NET_ratio", "num_detectors_ratio", "efficiency_ratio", "FOV_ratio"]]

In [ ]:
baseline_design["FOV_ratio"] = 1
baseline_design.loc["HFS1"]["FOV_ratio"] = 29.32/40.74
baseline_design.loc["HFS2"]["FOV_ratio"] = baseline_design.loc["HFS1"]["FOV_ratio"]

In [ ]:
del t.meta["comments"]


In [ ]:
t.columns

In [ ]:
for k in t.columns:
    if k != "band":
        t[k].format = "{:.3f}"

`N_ell_pdb / N_ell_dsr = (NET_pbd / NET_dsr * Eff_dsr / Eff_pdb * FOV_pdb/FOV_dsr) ** 2 * (ndet_dsr / ndet_pbd)`

In [ ]:
t["cl_ratio"] = (t["NET_ratio"] / t["efficiency_ratio"] * t["FOV_ratio"]) ** 2 / t["num_detectors_ratio"]

In [ ]:
t[["band", "cl_ratio"]]

In [ ]:
t.write("baseline_reference_ratio.dat", format="ascii.fixed_width")

In [ ]:
cat baseline_reference_ratio.dat